# Project 3: Đồ án Diagonalizable Matrix
#### Họ và tên: Trần Anh khoa
#### Lớp: 23CTT2
#### Mã số sinh viên: 23120135

## Yêu cầu 1: 
Cho $A$ là ma trận có thể chéo hóa được. Sinh viên viết chương trình tìm ma trận chéo $P$, $P^{-1}$ và ma trận đường chéo $D$, biết rằng $A = P D P^{-1}$.

## Trình bày về giải thuật chéo hóa

### Mục tiêu
Cho ma trận $A$ kích thước $n \times n$, thuật toán chéo hóa tìm ma trận đường chéo $D$ và ma trận khả nghịch $P$ sao cho $A = PDP^{-1}$, nếu $A$ chéo hóa được.

### Các bước thực hiện giải thuật

**Bước 1:** Tìm đa thức đặc trưng của ma trận A: 
$$
P_A(\lambda) = |A - \lambda I_n|.
$$
Nếu $P_A(\lambda)$ không có nghiệm trên $K$ thì A không chéo hóa được và thuật toán kết thúc. Ngược lại, chuyển sang **Bước 2**.

**Bước 2:** Tìm tất cả các trị riêng $\lambda_1, \lambda_2, \dots, \lambda_k$ của ma trận A từ việc giải phương trình đặc trưng, và với mỗi $\lambda_i$ như vậy ta cũng xác định được bội đại số tương ứng $\alpha_i$. Bên cạnh đó, với mỗi trị riêng ta tìm số chiều của không gian nghiệm $E(\lambda_i)$ bằng cách giải hệ phương trình 
$$
(A - \lambda_i I) \mathbf{x} = \vec{0}.
$$
Lúc này sẽ có 2 khả năng xảy ra:
- **Khả năng 1:** Nếu tồn tại một $i$ thuộc từ 1 đến $k$ thỏa $\text{dim}(E(\lambda_i)) \neq \alpha_i$ thì ma trận A không chéo hóa được và thuật toán kết thúc.
- **Khả năng 2:** Ngược lại với khả năng 1, A chéo hóa được và chuyển qua **Bước 3**.

**Bước 3:** Với mỗi $i$ thuộc từ 1 đến $k$, ta đi tìm một cơ sở $B_i$ cho không gian con $E(\lambda_i)$. Đặt $P$ là ma trận có được từ việc xếp lần lượt các vector trong $B_1, B_2, \dots, B_k$ thành các cột và đặt $D$ là ma trận đường chéo kích thước $n \times n$ với các phần tử trên đường chéo lần lượt là 
$
\underbrace{\lambda_1, \dots, \lambda_1}_{\alpha_1 \text{ lần}}, \dots, \underbrace{\lambda_k, \dots, \lambda_k}_{\alpha_k \text{ lần}}.
$

Khi đó:
$
A = P D P^{-1}
$
thỏa yêu cầu đề bài.


> ####  **LƯU Ý**
>
> - Thuật toán chéo hóa được trình bày ở trên là hoàn toàn hợp lý về mặt lý thuyết. Tuy nhiên, trong thực tế lập trình, việc tìm trị riêng và vector riêng cho các ma trận có kích thước lớn (từ $4 \times 4$ trở lên) là khó khăn và bất tiện. Vì vậy, trong phần mã nguồn của tôi ở dưới, với các ma trận kích thước **$2 \times 2$ và $3 \times 3$**, tôi sẽ thực hiện chéo hóa chính xác theo đúng thuật toán chéo hóa đã nêu ở trên, còn với các ma trận có kích thước từ **$4 \times 4$** trở lên, tôi sẽ **ứng dụng phân rã QR** (đã được học và lập trình ở các buổi lab trước) để hỗ trợ việc chéo hóa thuận tiện hơn.

Sau đây là mã nguồn thực hiện yêu cầu đề bài

In [ ]:
# ===================== CÁC HÀM PHỤ TRỢ =================================

# Hàm tạo ma trận có kích thước row x col mà các số hạng đều là số 0
def create_zero_matrix(row, col):
    return [[0 for _ in range(col)] for _ in range(row)]

# Hàm trích xuất cột của một ma trận
def extract_column(matrix, col_index):
    return [row[col_index] for row in matrix]

# Hàm tính chuẩn (norm) của vector
def norm(vec):
    result = 0
    for v in vec:
        result += v * v
    return result ** 0.5

# Hàm nhân một số với vector
def multiply_by_number(number, vector):
    result = []
    for element in vector:
        result.append(number * element)
    return result

# Hàm trừ hai vector
def subtract_vectors(vec1, vec2):
    if len(vec1) != len(vec2):
        raise ValueError("Hai vector cần có cùng độ dài")
    
    result = []
    for i in range(len(vec1)):
        result.append(vec1[i] - vec2[i])
    return result
    
# Hàm tính tích vô hướng của hai vector
def dot_product_vectors(vec1, vec2):
    if len(vec1) != len(vec2):
        raise ValueError("Hai vector phải có cùng độ dài")
    
    result = 0
    for i in range(len(vec1)):
        result += vec1[i] * vec2[i]
    return result

# Hàm nhân hai ma trận
def multiply_matrix(A_list, B_list):
    result_list = [[0 for _ in range(len(B_list[0]))] for _ in range(len(A_list))]

    m_row_A = len(A_list)
    n_col_B = len(B_list[0])
    for i_row in range(m_row_A):
        for i_col in range(n_col_B):
            total = 0
            for i,a in enumerate(A_list[i_row]): 
                total += a*B_list[i][i_col]
            result_list[i_row][i_col] = total

    return result_list

# Hàm chuyển vị ma trận
def transpose(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    result_matrix = create_zero_matrix(cols, rows)
    for i in range(rows):
        for j in range(cols):
            result_matrix[j][i] = matrix[i][j]
    return result_matrix

# Hàm tính định thức ma trận (cho ma trận bậc 1, bậc 2, bậc 3)
def determinant(matrix, n):
    if n == 1:
        return matrix[0][0]
    if n == 2:
        return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
    if n == 3:
        a11, a12, a13 = matrix[0]
        a21, a22, a23 = matrix[1]
        a31, a32, a33 = matrix[2]
        # Sử dụng quy tắc Sarrus
        positive = (a11 * a22 * a33) + (a12 * a23 * a31) + (a13 * a21 * a32)
        negative = (a13 * a22 * a31) + (a11 * a23 * a32) + (a12 * a21 * a33)
        return positive - negative

# Hàm in ma trận
def print_matrix(matrix, name="Matrix"):
    print(f"{name}:")
    for row in matrix:
        print([round(val, 8) if isinstance(val, float) else val for val in row])
    print()

# Hàm tạo ma trận đơn vị I kích thước n x n
def identity_matrix(n):
    I = []
    for i in range(n):
        row = []
        for j in range(n):
            if i == j:
                row.append(1)
            else:
                row.append(0)
        I.append(row)
    return I
    
# Trừ hai ma trận A - B
def subtract_matrices(A, B):
    result = []
    for i in range(len(A)):
        row = []
        for j in range(len(A[0])):
            row.append(A[i][j] - B[i][j])
        result.append(row)
    return result

# Nhân một số vô hướng với ma trận A 
def scalar_multiply_matrix(scalar, A):
    result = []
    for i in range(len(A)):
        row = []
        for j in range(len(A[0])):
            row.append(scalar * A[i][j])
        result.append(row)
    return result

# Hàm tìm ma trận nghịch đảo bằng phương pháp Gauss–Jordan
def find_inverse(matrix):
    size = len(matrix)

    # Tạo ma trận mở rộng [matrix | I]
    augmented = []
    for row_index in range(size):
        row = []
        
        for col_index in range(size):
            row.append(matrix[row_index][col_index])
            
        for col_index in range(size):
            if row_index == col_index:
                row.append(1)
            else:
                row.append(0)
        augmented.append(row)

    # Khử Gauss–Jordan
    for pivot_index in range(size):
        # Nếu phần tử chéo chính bằng 0 thì ta đổi dòng
        if augmented[pivot_index][pivot_index] == 0:
            found = False
            for swap_row in range(pivot_index + 1, size):
                if augmented[swap_row][pivot_index] != 0:
                    augmented[pivot_index], augmented[swap_row] = augmented[swap_row], augmented[pivot_index]
                    found = True
                    break
            if not found:
                print("Ma trận không khả nghịch")
                return None

        # Biến phần tử chéo chính thành 1
        pivot_value = augmented[pivot_index][pivot_index]
        for col in range(2 * size):
            augmented[pivot_index][col] /= pivot_value

        # Khử các phần tử khác trong cột để tạo giá trị 0
        for row_index in range(size):
            if row_index != pivot_index:
                factor = augmented[row_index][pivot_index]
                for col in range(2 * size):
                    augmented[row_index][col] -= factor * augmented[pivot_index][col]

    # Phần bên phải của ma trận lúc này là ma trận nghịch đảo cần tìm
    inverse = []
    for row_index in range(size):
        inverse_row = []
        for col_index in range(size, 2 * size):
            inverse_row.append(augmented[row_index][col_index])
        inverse.append(inverse_row)

    return inverse
    
# Hàm chuẩn hóa nghiệm của phương trình đa thức đặc trưng
def normalize_roots(roots, tol=1e-1):
    normalized = []
    for x in roots:
        matched = False
        for r in normalized:
            if abs(x - r) < tol:
                normalized.append(r)  # Gán về giá trị đã có
                matched = True
                break
        if not matched:
            # Làm tròn về số nguyên nếu gần số nguyên
            rounded = round(x)
            if abs(x - rounded) < tol:
                normalized.append(float(rounded))
            else:
                normalized.append(x)
    return normalized
    
# Hàm giải phương trình đa thức đặc trưng 
def solve_characteristic_polynomial(coeffs, n):
    EPS = 1e-10       # Sai số chấp nhận
    STEP = 0.05       # Bước nhảy dò nghiệm
    RANGE = 100       # Phạm vi tìm nghiệm
    DIGITS = 8        # Làm tròn sau bao nhiêu chữ số

    def eval_poly(coeffs, x):
        # Tính giá trị đa thức tại x
        result = 0
        for i, a in enumerate(reversed(coeffs)):
            result += a * (x ** i)
        return result

    def bisection(coeffs, a, b):
        # Tìm nghiệm trong đoạn [a, b] bằng phương pháp chia đôi
        fa = eval_poly(coeffs, a)
        fb = eval_poly(coeffs, b)
        if fa * fb > 0:
            return None  # Không có nghiệm trong khoảng
        for _ in range(100):
            mid = (a + b) / 2
            fmid = eval_poly(coeffs, mid)
            if abs(fmid) < EPS:
                return round(mid, DIGITS)
            if fa * fmid < 0:
                b = mid
                fb = fmid
            else:
                a = mid
                fa = fmid
        return round((a + b) / 2, DIGITS)

    def solve_quadratic(a, b, c):
        delta = b * b - 4 * a * c
        if delta < 0:
            return []  # Bỏ nghiệm phức
        sqrt_delta = delta ** 0.5
        x1 = (-b + sqrt_delta) / (2 * a)
        x2 = (-b - sqrt_delta) / (2 * a)
        return [round(x1, DIGITS), round(x2, DIGITS)]

    if n == 2:
        return solve_quadratic(*coeffs)

    elif n == 3:
        roots = []
        x = -RANGE
        while x <= RANGE:
            x_next = x + STEP
            y1 = eval_poly(coeffs, x)
            y2 = eval_poly(coeffs, x_next)
            if y1 * y2 <= 0:
                root = bisection(coeffs, x, x_next)
                if root is not None:
                    roots.append(root)  
            x = x_next
        return normalize_roots(roots)

# Hàm khử Gauss để giải (A - λI)x = (0, 0, 0) và tìm số chiều không gian riêng 
def gauss_elimination(matrix, n):
    A = [row[:] for row in matrix]
    rank = 0
    for col in range(n):
        pivot_row = None
        for row in range(rank, n):
            if abs(A[row][col]) > 1e-10:
                pivot_row = row
                break
        if pivot_row is None:
            continue
        A[pivot_row], A[rank] = A[rank], A[pivot_row]
        pivot = A[rank][col]
        for j in range(n):
            A[rank][j] /= pivot
        for i in range(n):
            if i != rank:
                factor = A[i][col]
                for j in range(n):
                    A[i][j] -= factor * A[rank][j]
        rank += 1
    return n - rank

# Hàm tìm vector riêng (cơ sở của không gian riêng) 
def find_eigenvectors(matrix, n):
    A = [row[:] for row in matrix]
    basis = []
    free_vars = []
    rank = 0
    for col in range(n):
        pivot_row = None
        for row in range(rank, n):
            if abs(A[row][col]) > 1e-10:
                pivot_row = row
                break
        if pivot_row is None:
            free_vars.append(col)
            continue
        A[pivot_row], A[rank] = A[rank], A[pivot_row]
        pivot = A[rank][col]
        for j in range(n):
            A[rank][j] /= pivot
        for i in range(n):
            if i != rank:
                factor = A[i][col]
                for j in range(n):
                    A[i][j] -= factor * A[rank][j]
        rank += 1
    for free_var in free_vars:
        vec = [0] * n
        vec[free_var] = 1
        for row in range(min(rank, n)):
            pivot_col = next(col for col in range(n) if abs(A[row][col]) > 1e-10)
            if pivot_col < free_var:
                vec[pivot_col] = -A[row][free_var]
        basis.append(vec)
    return basis

# Hàm tạo ma trận (A - lamda*I)
def matrix_minus_lambda_I(matrix, lam):
    n = len(matrix)  
    I = identity_matrix(n)  
    lambda_I = scalar_multiply_matrix(lam, I)  
    return subtract_matrices(matrix, lambda_I) 

# Hàm tính vết của ma trận 
def trace_matrix(M):
    return sum(M[i][i] for i in range(len(M)))
    
# Hàm tính hệ số cho đa thức đặc trưng của ma trận 2 x 2
def get_characteristic_polynomial_2x2(A):
    # Tính vết của A
    trace = trace_matrix(A)
    # Tính định thức của A
    determinant_val = determinant(A, 2)

    return [1, -trace, determinant_val] 

# Hàm nhân hai ma trận vuông (
def matrix_multiply(A, B):
    n = len(A)
    result = create_zero_matrix(n, n)
    for i in range(n):
        for j in range(n):
            for k in range(n):
                result[i][j] += A[i][k] * B[k][j]
    return result

# Hàm tính đa thức đặc trưng của ma trận 3x3 theo công thức Newton-Girard
def get_characteristic_polynomial_3x3(A):
    # Tính vết của A
    s1 = trace_matrix(A)

    # Tính tổng định thức các ma trận con 2x2 
    # M11: bỏ dòng 0, cột 0
    minor00 = A[1][1] * A[2][2] - A[1][2] * A[2][1]
    # M22: bỏ dòng 1, cột 1
    minor11 = A[0][0] * A[2][2] - A[0][2] * A[2][0]
    # M33: bỏ dòng 2, cột 2
    minor22 = A[0][0] * A[1][1] - A[0][1] * A[1][0]
    s2 = minor00 + minor11 + minor22

    # Tính định thức của ma trận 3x3
    s3 = determinant(A, 3)
    
    return [1, -s1, s2, -s3]

# Hàm chuẩn hóa ma trận để loại bỏ -0.0 và các số rất nhỏ
def normalize_matrix(matrix):
    row = len(matrix)
    col = len(matrix[0])
    
    for i in range(row):
        for j in range(col):
            # Kiểm tra số rất nhỏ 
            if abs(matrix[i][j]) < 1e-5:
                matrix[i][j] = 0.0
            # Kiểm tra và thay đổi các số -0.0
            if matrix[i][j] == -0.0:
                matrix[i][j] = 0.0
    
    return matrix

# Hàm kiểm tra hai ma trận có bằng nhau không (với ngưỡng sai số cho phép)
def check_matrix_similarity(matrix1, matrix2, e = 1e-7):
    if len(matrix1) != len(matrix2) or len(matrix1[0]) != len(matrix2[0]):
        return False
    for i in range(len(matrix1)):
        for j in range(len(matrix1[0])):
            if abs(matrix1[i][j] - matrix2[i][j]) > e:
                return False
    return True

# ================================== HÀM CHÉO HÓA MA TRẬN (DÙNG CHO MA TRẬN 2x2 và 3x3) ====================================
def diagonalize_matrix(A):
    n = len(A)
    
    # Tính đa thức đặc trưng và nghiệm (giá trị riêng)
    if n == 2:
        coeffs = get_characteristic_polynomial_2x2(A)
    elif n == 3:
        coeffs = get_characteristic_polynomial_3x3(A)
    else:
        print("Chỉ hỗ trợ ma trận 2x2 hoặc 3x3.")
        return None

    eigenvalues = solve_characteristic_polynomial(coeffs, n)
    if not eigenvalues:
        print("Ma trận không chéo hóa được vì không có nghiệm thực.")
        return None

    # Tính bội đại số (algebraic multiplicity) bằng cách đếm số lần xuất hiện
    unique_vals = []
    mult_counts = []

    for val in eigenvalues:
        matched = False
        for i in range(len(unique_vals)):
            if abs(val - unique_vals[i]) < 1e-8:
                mult_counts[i] += 1
                matched = True
                break
        if not matched:
            unique_vals.append(val)
            mult_counts.append(1)

    # Bước 2: Kiểm tra điều kiện chéo hóa và tìm vector riêng
    eigenvectors_all = []
    diag_elements = []

    for idx in range(len(unique_vals)):
        lam = unique_vals[idx]
        multiplicity = mult_counts[idx]

        A_minus_lamI = matrix_minus_lambda_I(A, lam)
        vectors = find_eigenvectors(A_minus_lamI, n)
        geom_mult = len(vectors)

        if geom_mult < multiplicity:
            print(f"Không chéo hóa được vì dim(E({lam})) < bội đại số ({multiplicity})")
            return None

        # Lưu các vector riêng (chỉ lấy đúng số lượng cần)
        for i in range(multiplicity):
            eigenvectors_all.append(vectors[i])
            diag_elements.append(lam)

    # Bước 3: Tạo ma trận P và D
    # - P: các vector riêng là cột
    # - D: ma trận chéo với giá trị riêng trên đường chéo

    # Dựng P với mỗi vector riêng làm cột
    P = []
    for i in range(n):
        row = []
        for vec in eigenvectors_all:
            row.append(vec[i])
        P.append(row)

    P_inv = find_inverse(P)
    if P_inv is None:
        print("Không tìm được P⁻¹, không thể chéo hóa.")
        return None

    D = create_zero_matrix(n, n)
    for i in range(n):
        D[i][i] = diag_elements[i]

    return P, D, P_inv


# ========================== SỬ DỤNG PHÂN RÃ QR ĐỂ CHÉO HÓA CÁC MA TRẬN LỚN (TỪ 4x4 TRỞ LÊN) ==============================
# Hàm phân rã QR 
def qr_decomposition(A):
    m = len(A)  
    n = len(A[0]) 

    # Khởi tạo ma trận Q và ma trận R
    Q = create_zero_matrix(m, n) 
    R = create_zero_matrix(n, n) 

    # List các cột của ma trận A
    a_vectors = [extract_column(A, j) for j in range(n)]
    
    # Khởi tạo list lưu trữ vector trực chuẩn e_i
    e_vectors = []  

# Quá trình thực hiện thuật toán Gram-Schmidt
    
    for i in range(n):
        # Bước lấy f_i
        if i == 0:
            # f_1 chính là a_1
            f_i = a_vectors[i]
        else:
            # Với i >= 1, f_i được tính từ a_i và các e_j trước đó
            f_i = a_vectors[i]
            
            for j in range(i):
                proj = dot_product_vectors(a_vectors[i], e_vectors[j])
                f_i = subtract_vectors(f_i, multiply_by_number(proj, e_vectors[j]))
        
        # Bước 2: Tính e_i từ f_i
        norm_fi = norm(f_i)
        if norm_fi == 0:
            raise ValueError("Các cột của ma trận không độc lập tuyến tính")
        e_i = multiply_by_number(1 / norm_fi, f_i)
        e_vectors.append(e_i)

        # Cập nhật cột i của ma trận Q
        for row in range(m):
            Q[row][i] = e_i[row]

    # Xây dựng ma trận R
    for i in range(n):
        for j in range(i, n):
            R[i][j] = dot_product_vectors(a_vectors[j], e_vectors[i])

    return Q, R

# Hàm chéo hóa ma trận ứng dụng từ phân rã QR
def diagonalize_matrix_qr(A, max_iter=100, tol=1e-9):
    n = len(A)
    Ak = [row[:] for row in A]  # Lấy bản sao của A
    P_total = identity_matrix(n)  # Khởi tạo giá trị ban đầu P = I

    for _ in range(max_iter):
        Q, R = qr_decomposition(Ak)
        Ak_next = multiply_matrix(R, Q)

        # Tích lũy P (Q1 * Q2 * ... * Qk)
        P_total = multiply_matrix(P_total, Q)

        # Kiểm tra hội tụ: nếu phần dưới đường chéo gần 0 thì hội tụ
        off_diag_sum = 0
        for i in range(1, n):
            for j in range(i):
                off_diag_sum += abs(Ak_next[i][j])
        if off_diag_sum < tol:
            break

        Ak = Ak_next

    # Ma trận D gần đúng là Ak sau lặp
    D = normalize_matrix(Ak)
    P = normalize_matrix(P_total)
    P_inv = find_inverse(P)

    return P, D, P_inv

# ================================================ HÀM CHÉO HÓA MA TRẬN TỔNG QUÁT ====================================
def diagonalize_matrix_general(A):
    n = len(A)
    if n == 2 or n == 3:
        return diagonalize_matrix(A)
    elif n >= 4:
        return diagonalize_matrix_qr(A)
        
# ================================================== HÀM DÙNG ĐỂ KIỂM TRA CHÉO HÓA ====================================
def test_diagonalization(matrix, name="Ma trận"):
    print("-" * 50)
    print(f"Kiểm tra {name} (kích thước {len(matrix)}x{len(matrix[0])})")
    print("-" * 50)
    print_matrix(matrix, f"{name} ban đầu")

    result = diagonalize_matrix_general(matrix)

    if result:
        P, D, P_inv = result

        print_matrix(P, "Ma trận P")
        print_matrix(D, "Ma trận D")
        print_matrix(P_inv, "Ma trận P⁻¹")

        PD = matrix_multiply(P, D)
        PDP_inv = normalize_matrix(matrix_multiply(PD, P_inv))

        print_matrix(PDP_inv, "Ma trận PDP⁻¹ (khôi phục A)")

        if check_matrix_similarity(PDP_inv, matrix, 1e-6):
            print("A = P·D·P⁻¹ => Chéo hóa thành công.\n")
        else:
            print("A ≠ P·D·P⁻¹ => Chéo hóa không chính xác.\n")
    else:
        print("Không thể chéo hóa ma trận này.\n")

# ====================================================== THỬ NGHIỆM =======================================================

#Ma trận A dùng để chéo hóa
A_2x2 = [
    [1, 2],
    [-1, 4]
]

A_3x3 = [
    [1, -4, -4],
    [8, -11, -8],
    [-8, 8, 5]
]

A_4x4 = [
    [4, 1, 2, 2],
    [1, 3, 3, 1],
    [2, 3, 4, 3],
    [2, 1, 3, 4]
]

A_5x5 = [
    [5, 4, 1, 0, 0],
    [4, 5, 4, 1, 0],
    [1, 4, 5, 4, 1],
    [0, 1, 4, 5, 4],
    [0, 0, 1, 4, 5]
]

test_diagonalization(A_2x2, "A_2x2")
test_diagonalization(A_3x3, "A_3x3")
test_diagonalization(A_4x4, "A_4x4")
test_diagonalization(A_5x5, "A_5x5")

: 

## Mô tả các hàm

### 1. Hàm `create_zero_matrix(row, col)`
- **Tác dụng**: Tạo một ma trận có kích thước `$ \text{row} \times \text{col} $` với tất cả phần tử đều bằng 0.
- **Input**:
  - `row`: Số hàng của ma trận.
  - `col`: Số cột của ma trận.
- **Nguyên lý hoạt động**:
  - Dùng list comprehension lồng nhau để tạo danh sách các danh sách (ma trận).
  - Mỗi hàng gồm `col` phần tử có giá trị 0.
- **Output**: Ma trận `$ \text{row} \times \text{col} $` với tất cả phần tử bằng 0.

---

### 2. Hàm `extract_column(matrix, col_index)`
- **Tác dụng**: Trích xuất một vector cột từ ma trận cho trước.
- **Input**:
  - `matrix`: Ma trận đầu vào.
  - `col_index`: Chỉ số cột cần trích xuất.
- **Nguyên lý hoạt động**:
  - Duyệt qua từng hàng của ma trận.
  - Lấy phần tử tại cột `col_index` từ mỗi hàng.
- **Output**: Danh sách chứa các phần tử của cột được chọn.

---

### 3. Hàm `norm(vec)`
- **Tác dụng**: Tính chuẩn (norm) của một vector.
- **Input**: Một vector `vec`.
- **Nguyên lý hoạt động**:
  - Cộng bình phương các phần tử của vector.
  - Trả về căn bậc hai của tổng đó.
- **Output**: Chuẩn (magnitude) của vector, là một số thực.

---

### 4. Hàm `multiply_by_number(number, vector)`
- **Tác dụng**: Nhân một số vô hướng với một vector.
- **Input**:
  - `number`: Hệ số vô hướng.
  - `vector`: Vector cần nhân.
- **Nguyên lý hoạt động**:
  - Duyệt từng phần tử của vector và nhân với `number`.
- **Output**: Vector mới sau khi nhân với số.

---

### 5. Hàm `subtract_vectors(vec1, vec2)`
- **Tác dụng**: Thực hiện phép trừ hai vector cùng chiều.
- **Input**:
  - `vec1`: Vector thứ nhất.
  - `vec2`: Vector thứ hai.
- **Nguyên lý hoạt động**:
  - Kiểm tra độ dài hai vector có bằng nhau không.
  - Trừ từng phần tử tương ứng.
- **Output**: Vector kết quả sau phép trừ.

---

### 6. Hàm `dot_product_vectors(vec1, vec2)`
- **Tác dụng**: Tính tích vô hướng (dot product) giữa hai vector.
- **Input**:
  - `vec1`, `vec2`: Hai vector cùng chiều.
- **Nguyên lý hoạt động**:
  - Kiểm tra độ dài hai vector.
  - Tính tổng của các tích từng phần tử tương ứng.
- **Output**: Một số thực là tích vô hướng.

---

### 7. Hàm `multiply_matrix(A_list, B_list)`
- **Tác dụng**: Tính tích hai ma trận $A$ và $B$.
- **Input**:
  - `A_list`: Ma trận $A$.
  - `B_list`: Ma trận $B$.
- **Nguyên lý hoạt động**:
  - Duyệt từng cặp hàng-cột để tính tổng tích các phần tử tương ứng.
- **Output**: Ma trận kết quả sau khi nhân.

---

### 8. Hàm `transpose(matrix)`
- **Tác dụng**: Tính ma trận chuyển vị.
- **Input**: Ma trận `matrix`.
- **Nguyên lý hoạt động**:
  - Đổi vị trí hàng thành cột và ngược lại.
- **Output**: Ma trận chuyển vị của đầu vào.

---

### 9. Hàm `determinant(matrix, n)`
- **Tác dụng**: Tính định thức của ma trận bậc 1, 2 hoặc 3.
- **Input**:
  - `matrix`: Ma trận vuông.
  - `n`: Kích thước (bậc) của ma trận.
- **Nguyên lý hoạt động**:
  - Với bậc 2 dùng công thức cổ điển.
  - Với bậc 3 dùng quy tắc Sarrus.
- **Output**: Định thức của ma trận.

---

### 10. Hàm `identity_matrix(n)`
- **Tác dụng**: Tạo ma trận đơn vị kích thước `$ n \times n $`.
- **Input**: 
  - `n`: Kích thước cạnh của ma trận.
- **Nguyên lý hoạt động**:
  - Gán giá trị 1 ở đường chéo chính, còn lại là 0.
- **Output**: Ma trận đơn vị kích thước $n$.

---
### 11. Hàm `subtract_matrices(A, B)`
- **Tác dụng**: Thực hiện phép trừ hai ma trận cùng kích thước.
- **Input**:
  - `A`: Ma trận thứ nhất.
  - `B`: Ma trận thứ hai.
- **Nguyên lý hoạt động**:
  - Duyệt từng cặp phần tử tương ứng tại cùng chỉ số $(i, j)$ trong hai ma trận.
  - Tính hiệu $A[i][j] - B[i][j]$ và lưu vào ma trận kết quả.
- **Output**: Ma trận kết quả có cùng kích thước với `A` và `B`.

---

### 12. Hàm `scalar_multiply_matrix(scalar, A)`
- **Tác dụng**: Nhân một số vô hướng với tất cả các phần tử của một ma trận.
- **Input**:
  - `scalar`: Số thực hoặc nguyên cần nhân.
  - `A`: Ma trận cần nhân với `scalar`.
- **Nguyên lý hoạt động**:
  - Duyệt từng phần tử trong ma trận.
  - Nhân phần tử đó với `scalar` và ghi lại vào ma trận kết quả.
- **Output**: Ma trận mới có cùng kích thước với `A`, các phần tử đã được nhân với `scalar`.

---

### 13. Hàm `find_inverse(matrix)`
- **Tác dụng**: Tìm ma trận nghịch đảo bằng phương pháp Gauss–Jordan.
- **Input**:
  - `matrix`: Ma trận vuông cần tìm nghịch đảo.
- **Nguyên lý hoạt động**:
  - Ghép ma trận đầu vào với ma trận đơn vị thành ma trận mở rộng $[A \,|\, I]$.
  - Thực hiện phép khử Gauss–Jordan trên ma trận mở rộng:
    - Biến phần tử đường chéo chính thành 1.
    - Biến các phần tử còn lại trong cột thành 0.
    - Nếu phần tử đường chéo chính bằng 0 thì đổi dòng để tránh chia cho 0.
  - Sau khi khử hoàn toàn, nửa phải của ma trận mở rộng chính là nghịch đảo.
- **Output**: Ma trận nghịch đảo của `matrix`. Nếu không khả nghịch, trả về `None`.

---

### 14. Hàm `normalize_roots(roots, tol=1e-1)`
- **Tác dụng**: Chuẩn hóa nghiệm của phương trình đa thức đặc trưng để loại bỏ nghiệm trùng hoặc gần nhau.
- **Input**:
  - `roots`: Danh sách các nghiệm thực thu được từ quá trình giải phương trình.
  - `tol`: Ngưỡng sai số để xác định hai nghiệm "gần giống nhau" (mặc định là $1e{-1}$).
- **Nguyên lý hoạt động**:
  - Duyệt qua từng nghiệm trong `roots`.
  - Nếu nghiệm gần bằng với một nghiệm đã tồn tại trong danh sách chuẩn hóa thì dùng lại nghiệm cũ.
  - Nếu nghiệm gần một số nguyên thì làm tròn về số nguyên đó.
- **Output**: Danh sách các nghiệm đã được chuẩn hóa, loại bỏ nhiễu số nhỏ và trùng lặp gần.

---

### 15. Hàm `solve_characteristic_polynomial(coeffs, n)`
- **Tác dụng**: Giải phương trình đa thức đặc trưng để tìm các trị riêng của ma trận.
- **Input**:
  - `coeffs`: Danh sách hệ số của đa thức đặc trưng.
  - `n`: Bậc của đa thức (chỉ hỗ trợ bậc 2 và 3).
- **Nguyên lý hoạt động**:
  - Với bậc 2: dùng công thức nghiệm tổng quát.
  - Với bậc 3: sử dụng phương pháp chia đôi (bisection method) để tìm nghiệm trong đoạn $[-100, 100]$ với bước nhảy nhỏ.
  - Kiểm tra từng đoạn có dấu đổi để áp dụng chia đôi.
  - Chuẩn hóa nghiệm sau khi tìm được để loại bỏ nghiệm gần nhau hoặc nhiễu số.
- **Output**: Danh sách các trị riêng (nghiệm thực) của đa thức đặc trưng.

---

### 16. Hàm `gauss_elimination(matrix, n)`
- **Tác dụng**: Thực hiện khử Gauss để tìm hạng (rank) của ma trận, hỗ trợ tính số chiều không gian riêng.
- **Input**:
  - `matrix`: Ma trận hệ số tương ứng với hệ phương trình đồng nhất.
  - `n`: Số biến (kích thước ma trận vuông).
- **Nguyên lý hoạt động**:
  - Thực hiện khử Gauss bằng cách biến đổi ma trận về dạng bậc thang.
  - Đếm số hàng khác không để xác định hạng của ma trận.
  - Số chiều không gian riêng sẽ bằng $n - \text{rank}$.
- **Output**: Số chiều không gian riêng (số nghiệm tự do của hệ).

---

### 17. Hàm `find_eigenvectors(matrix, n)`
- **Tác dụng**: Tìm cơ sở của không gian riêng ứng với một trị riêng đã biết, tức là tìm các vector riêng.
- **Input**:
  - `matrix`: Ma trận dạng $A - \lambda I$, trong đó $\lambda$ là trị riêng.
  - `n`: Kích thước của ma trận.
- **Nguyên lý hoạt động**:
  - Thực hiện khử Gauss để đưa ma trận về dạng bậc thang hàng rút gọn (RREF).
  - Tìm các biến tự do trong hệ phương trình đồng nhất tương ứng.
  - Gán giá trị 1 cho từng biến tự do để tạo ra các vector cơ sở độc lập.
  - Với mỗi biến tự do, tính giá trị các biến còn lại bằng cách thế ngược từ ma trận bậc thang.
- **Output**: Danh sách các vector cơ sở của không gian riêng, tức là các vector riêng ứng với trị riêng đầu vào.

---

### 18. Hàm `matrix_minus_lambda_I(matrix, lam)`
- **Tác dụng**: Tính hiệu của ma trận $A$ và $\lambda I$, dùng để phục vụ cho việc tìm không gian riêng.
- **Input**:
  - `matrix`: Ma trận vuông $A$.
  - `lam`: Trị riêng $\lambda$ (số thực).
- **Nguyên lý hoạt động**:
  - Tạo ma trận đơn vị $I$ có cùng kích thước với $A$.
  - Nhân $\lambda$ với $I$ để được $\lambda I$.
  - Trừ $\lambda I$ khỏi $A$ để được ma trận $A - \lambda I$.
- **Output**: Ma trận $A - \lambda I$.

---

### 19. Hàm `trace_matrix(M)`
- **Tác dụng**: Tính vết (trace) của một ma trận vuông.
- **Input**:
  - `M`: Ma trận vuông.
- **Nguyên lý hoạt động**:
  - Duyệt qua đường chéo chính của ma trận.
  - Cộng tất cả các phần tử nằm trên đường chéo chính.
- **Output**: Một số thực là tổng các phần tử trên đường chéo chính (vết của ma trận).

---

### 20. Hàm `get_characteristic_polynomial_2x2(A)`
- **Tác dụng**: Tính hệ số của đa thức đặc trưng bậc 2 từ ma trận $A$ kích thước $2 \times 2$.
- **Input**:
  - `A`: Ma trận vuông kích thước $2 \times 2$.
- **Nguyên lý hoạt động**:
  - Dựa vào công thức đặc trưng:  
    $\det(A - \lambda I) = \lambda^2 - \text{Tr}(A)\lambda + \det(A)$
  - Tính vết và định thức của $A$ rồi đưa vào công thức trên.
- **Output**: Danh sách hệ số của đa thức đặc trưng theo thứ tự: $[1, -\text{trace}, \text{det}]$.

### 21. Hàm `matrix_multiply(A, B)`
- **Tác dụng**: Thực hiện phép nhân hai ma trận vuông cùng kích thước.
- **Input**:
  - `A`: Ma trận vuông kích thước $n \times n$.
  - `B`: Ma trận vuông kích thước $n \times n$.
- **Nguyên lý hoạt động**:
  - Với mỗi phần tử $C[i][j]$ trong kết quả, tính bằng tổng các tích $A[i][k] \cdot B[k][j]$ với $k$ chạy từ $0$ đến $n - 1$.
- **Output**: Ma trận tích $C = AB$ có cùng kích thước $n \times n$.

---

### 22. Hàm `get_characteristic_polynomial_3x3(A)`
- **Tác dụng**: Tính hệ số của đa thức đặc trưng bậc 3 từ ma trận $A$ kích thước $3 \times 3$.
- **Input**:
  - `A`: Ma trận vuông kích thước $3 \times 3$.
- **Nguyên lý hoạt động**:
  - Tính vết $s_1 = \text{Tr}(A)$.
  - Tính tổng định thức của các ma trận con $2 \times 2$ nằm trên đường chéo chính (được dùng để tính $s_2$).
  - Tính định thức $s_3 = \det(A)$.
  - Đưa vào công thức:
    $$
    \det(A - \lambda I) = \lambda^3 - s_1 \lambda^2 + s_2 \lambda - s_3
    $$
- **Output**: Danh sách hệ số của đa thức đặc trưng bậc 3: $[1, -s_1, s_2, -s_3]$.

---

### 23. Hàm `normalize_matrix(matrix)`
- **Tác dụng**: Chuẩn hóa ma trận để loại bỏ các số rất nhỏ gần 0 và các giá trị $-0.0$.
- **Input**:
  - `matrix`: Ma trận cần chuẩn hóa.
- **Nguyên lý hoạt động**:
  - Duyệt từng phần tử trong ma trận:
    - Nếu giá trị tuyệt đối nhỏ hơn ngưỡng (mặc định $1e^{-5}$), gán về $0.0$.
    - Nếu giá trị là $-0.0$, cũng gán về $0.0$.
- **Output**: Ma trận đã được làm sạch nhiễu số.

---

### 24. Hàm `check_matrix_similarity(matrix1, matrix2, e=1e-7)`
- **Tác dụng**: So sánh hai ma trận có bằng nhau hay không trong một ngưỡng sai số cho phép.
- **Input**:
  - `matrix1`: Ma trận thứ nhất.
  - `matrix2`: Ma trận thứ hai.
  - `e`: Ngưỡng sai số cho phép (mặc định là $1e^{-7}$).
- **Nguyên lý hoạt động**:
  - So sánh kích thước hai ma trận.
  - Duyệt từng phần tử tương ứng và kiểm tra sai lệch tuyệt đối có nhỏ hơn $e$ không.
- **Output**: `True` nếu hai ma trận "gần như bằng nhau", `False` nếu không.

---

### 25. Hàm `diagonalize_matrix(A)`
- **Tác dụng**: Chéo hóa chính xác ma trận vuông kích thước $2 \times 2$ hoặc $3 \times 3$ bằng cách tìm trị riêng và vector riêng.
- **Input**:
  - `A`: Ma trận vuông có kích thước $2 \times 2$ hoặc $3 \times 3$.
- **Nguyên lý hoạt động**:
  - **Bước 1**: Tính đa thức đặc trưng của ma trận $A$:
    - Nếu $n = 2$, dùng công thức trực tiếp.
    - Nếu $n = 3$, dùng định lý Newton–Girard.
  - **Bước 2**: Giải đa thức đặc trưng để tìm các trị riêng (nghiệm thực).
  - **Bước 3**: Tính bội đại số của từng trị riêng bằng cách đếm số lần xuất hiện.
  - **Bước 4**: Với mỗi trị riêng:
    - Tính ma trận $A - \lambda I$.
    - Tìm các vector riêng tương ứng bằng khử Gauss.
    - Kiểm tra số chiều không gian riêng (bội hình học) có đủ so với bội đại số hay không.
    - Nếu không đủ, kết luận không chéo hóa được.
  - **Bước 5**: Dựng ma trận $P$ (các vector riêng làm cột) và ma trận $D$ (các trị riêng trên đường chéo).
  - **Bước 6**: Tính $P^{-1}$ bằng phương pháp Gauss–Jordan.
- **Output**: Bộ ba ma trận $(P, D, P^{-1})$ sao cho $A = P D P^{-1}$, hoặc `None` nếu không chéo hóa được.

---

### 26. Hàm `qr_decomposition(A)`
- **Tác dụng**: Phân rã một ma trận $A$ thành tích của hai ma trận $Q$ và $R$, trong đó $Q$ trực chuẩn và $R$ tam giác trên.
- **Input**:
  - `A`: Ma trận kích thước $m \times n$ cần phân rã.
- **Nguyên lý hoạt động**:
  - Thực hiện thuật toán Gram–Schmidt để tạo ra các vector trực giao $f_i$, sau đó chuẩn hóa thành các vector đơn vị $e_i$.
  - Các vector $e_i$ được xếp thành các cột của ma trận $Q$.
  - Ma trận $R$ được xây dựng từ các tích vô hướng giữa $a_j$ và $e_i$.
- **Output**: Hai ma trận $(Q, R)$ sao cho $A = QR$, với $Q$ trực chuẩn và $R$ tam giác trên.

---

### 27. Hàm `diagonalize_matrix_qr(A, max_iter=100, tol=1e-9)`
- **Tác dụng**: Chéo hóa gần đúng ma trận vuông (kích thước $\geq 4 \times 4$) bằng phương pháp lặp QR Algorithm.
- **Input**:
  - `A`: Ma trận vuông cần chéo hóa.
  - `max_iter`: Số lần lặp tối đa (mặc định là 100).
  - `tol`: Ngưỡng sai số để kiểm tra hội tụ (mặc định là $1e^{-9}$).
- **Nguyên lý hoạt động**:
  - Khởi tạo $A_0 = A$ và $P = I$.
  - Trong mỗi vòng lặp:
    - Phân rã QR: $A_k = Q_k R_k$.
    - Tính $A_{k+1} = R_k Q_k$.
    - Cập nhật tích $P = P \cdot Q_k$ để tích lũy ma trận chuyển cơ sở.
    - Kiểm tra điều kiện hội tụ: nếu phần dưới đường chéo của $A_k$ gần bằng 0 thì dừng.
  - Sau khi hội tụ:
    - $D$ là ma trận gần tam giác (gần chéo).
    - $P$ là tích các ma trận $Q_k$.
    - $P^{-1}$ được tính bằng phương pháp Gauss–Jordan.
- **Output**: Bộ ba ma trận $(P, D, P^{-1})$ sao cho gần đúng $A \approx P D P^{-1}$.

---

### 28. Hàm `diagonalize_matrix_general(A)`
- **Tác dụng**: Hàm tổng quát để chéo hóa ma trận bất kỳ (nếu có thể), tự động chọn phương pháp phù hợp.
- **Input**:
  - `A`: Ma trận vuông kích thước $n \times n$.
- **Nguyên lý hoạt động**:
  - Kiểm tra kích thước $n$ của ma trận:
    - Nếu $n = 2$ hoặc $n = 3$, dùng hàm `diagonalize_matrix()` để chéo hóa chính xác.
    - Nếu $n \geq 4$, dùng `diagonalize_matrix_qr()` để chéo hóa gần đúng bằng phương pháp lặp QR.
- **Output**: Bộ ba ma trận $(P, D, P^{-1})$ nếu chéo hóa được, hoặc `None` nếu không thể.

---

### 29. Hàm `test_diagonalization(matrix, name="Ma trận")`
- **Tác dụng**: Hàm dùng để kiểm tra chéo hóa ma trận có đúng không bằng cách thực hiện $P D P^{-1}$ và so sánh với $A$.
- **Input**:
  - `matrix`: Ma trận đầu vào cần kiểm tra.
  - `name`: Tên gán cho ma trận (mặc định là "Ma trận").
- **Nguyên lý hoạt động**:
  - In thông tin và kích thước ma trận đầu vào.
  - Gọi hàm `diagonalize_matrix_general()` để chéo hóa.
  - Nếu thành công:
    - In các ma trận $P$, $D$, $P^{-1}$.
    - Tính $P D P^{-1}$ và chuẩn hóa kết quả.
    - So sánh với ma trận gốc bằng hàm `check_matrix_similarity()`.
  - In kết luận chéo hóa thành công hay không.
- **Output**: Không trả về giá trị. Chỉ in kết quả kiểm tra ra màn hình.


## Yêu cầu 2:
### Kiểm tra kết quả với hàm có sẵn
Trong phần này ta sẽ thực hiện việc kiểm tra kết quả của hàm chéo hóa ma trận do tôi tự xây dựng ở các yêu cầu trước với hàm có sẵn trong thư viện của Python. Sau đây là đoạn mã nguồn sử dụng thư viện **NumPy** và một hàm trong thư viện đó là **np.linalg.eig()** để thực hiện việc chéo hóa ma trận.

In [ ]:
import numpy as np

def diagonalize_with_numpy(A, name="Ma trận"):
    A = np.array(A, dtype=float)

    print("-" * 50)
    print(f"Kiểm tra {name} (kích thước {A.shape[0]}x{A.shape[1]})")
    print("-" * 50)

    # 1. Tính trị riêng và vector riêng
    eigenvalues, eigenvectors = np.linalg.eig(A)

    # 2. Dựng ma trận D và P
    D = np.diag(eigenvalues)
    P = eigenvectors
    try:
        P_inv = np.linalg.inv(P)
    except np.linalg.LinAlgError:
        print("Không thể nghịch đảo ma trận P. Không chéo hóa được.\n")
        return

    # 3. Khôi phục A để kiểm tra
    A_reconstructed = P @ D @ P_inv

    # 4. In kết quả
    np.set_printoptions(precision=6, suppress=True)
    print("Ma trận A:")
    print(A)
    print("\nMa trận P:")
    print(P)
    print("\nMa trận D:")
    print(D)
    print("\nMa trận P⁻¹:")
    print(P_inv)
    print("\nMa trận P·D·P⁻¹:")
    print(A_reconstructed)

    # 5. Kiểm tra chính xác
    if np.allclose(A, A_reconstructed, atol=1e-6):
        print("\nKẾT LUẬN: Chéo hóa THÀNH CÔNG (A ≈ P·D·P⁻¹)\n")
    else:
        print("\nKẾT LUẬN: Chéo hóa KHÔNG CHÍNH XÁC (do sai số hoặc trị riêng phức)\n")

A_2x2 = [
    [1, 2],
    [-1, 4]
]

A_3x3 = [
    [1, -4, -4],
    [8, -11, -8],
    [-8, 8, 5]
]

A_4x4 = [
    [4, 1, 2, 2],
    [1, 3, 3, 1],
    [2, 3, 4, 3],
    [2, 1, 3, 4]
]

A_5x5 = [
    [5, 4, 1, 0, 0],
    [4, 5, 4, 1, 0],
    [1, 4, 5, 4, 1],
    [0, 1, 4, 5, 4],
    [0, 0, 1, 4, 5]
]

diagonalize_with_numpy(A_2x2, "A_2x2")
diagonalize_with_numpy(A_3x3, "A_3x3")
diagonalize_with_numpy(A_4x4, "A_4x4")
diagonalize_with_numpy(A_5x5, "A_5x5")

Ta nhận thấy rằng, với cùng một ma trận **A**, kết quả của việc chéo hóa ma trận được thực hiện bởi chương trình tự xây dựng của tôi và hàm có sẵn trong Python là giống nhau. (Một số sai khác nhỏ về **dấu** hoặc **thứ tự vector riêng** là điều bình thường trong quá trình chéo hóa và không làm thay đổi bản chất của kết quả. Điều này **không ảnh hưởng** đến việc tái tạo lại ma trận **A** khi thực hiện phép nhân **P** $\times$ **D** $\times$ **P⁻¹**).

### Ứng dụng của chéo hóa ma trận

#### **1. Tính lũy thừa của ma trận**

##### **Bài toán**  
Cho $A \in M_n(\mathbb{R})$ và giả sử $A$ chéo hóa được, tức tồn tại ma trận khả nghịch $P$ và ma trận đường chéo $D$ sao cho:
$$
A = P D P^{-1}
$$
Tính $A^k$ với $k \in \mathbb{N}$.

##### **Mô tả tổng quát**  
Việc tính lũy thừa $A^k$ thông qua phép nhân ma trận thông thường đòi hỏi nhiều phép toán và dễ gây sai số. Tuy nhiên, nếu $A$ chéo hóa được, ta có thể tính $A^k$ một cách hiệu quả hơn bằng công thức:
$$
A^k = (P D P^{-1})^k = P D^k P^{-1}
$$
Vì $D$ là ma trận đường chéo, nên $D^k$ được tính dễ dàng bằng cách lũy thừa từng phần tử trên đường chéo.

##### **Cách áp dụng**  
1. Tìm ma trận $P$, $D$, $P^{-1}$ sao cho $A = P D P^{-1}$.
2. Tính $D^k$ bằng cách nâng từng phần tử đường chéo của $D$ lên lũy thừa $k$.
3. Tính $A^k = P D^k P^{-1}$.

##### **Ứng dụng thực tế**  
- **Mô phỏng động lực học rời rạc**: Tính trạng thái hệ thống sau $k$ bước biến đổi.
- **Phương trình vi phân tuyến tính**: Biểu diễn nghiệm của hệ bằng $e^{At}$ thông qua chuỗi lũy thừa.
- **Kỹ thuật số và xử lý tín hiệu**: Tính toán hiệu quả các phép biến đổi tuyến tính lặp lại.

Chéo hóa giúp đơn giản hóa quá trình tính toán lũy thừa của ma trận, đặc biệt với các hệ thống lớn và cần tính toán nhanh, chính xác.

---

#### **2. Tìm một dãy số thỏa công thức truy hồi**

##### **Bài toán**  
Cho một dãy số $(u_n)$ thỏa mãn công thức truy hồi tuyến tính bậc $k$:
$$
u_{n+k} = a_{k-1} u_{n+k-1} + a_{k-2} u_{n+k-2} + \cdots + a_0 u_n
$$
với các giá trị ban đầu $u_0, u_1, \ldots, u_{k-1}$ đã biết. Tìm công thức tổng quát cho $u_n$.

##### **Mô tả tổng quát**  
Dãy $(u_n)$ có thể được biểu diễn dưới dạng hệ phương trình ma trận. Ta định nghĩa vector trạng thái:
$$
\mathbf{U}_n = \begin{bmatrix}
u_{n+k-1} \\
u_{n+k-2} \\
\vdots \\
u_n
\end{bmatrix}
$$
Khi đó tồn tại một ma trận $A \in M_k(\mathbb{R})$ sao cho:
$$
\mathbf{U}_{n+1} = A \mathbf{U}_n
$$
Nếu $A$ chéo hóa được, tức tồn tại $P$, $D$, $P^{-1}$ sao cho $A = P D P^{-1}$, thì ta có:
$$
\mathbf{U}_n = A^n \mathbf{U}_0 = P D^n P^{-1} \mathbf{U}_0
$$

##### **Cách áp dụng**  
1. Xây dựng ma trận truy hồi $A$ từ hệ số $a_i$.
2. Chéo hóa $A$: tìm $P$, $D$, $P^{-1}$ sao cho $A = P D P^{-1}$.
3. Tính $\mathbf{U}_n = P D^n P^{-1} \mathbf{U}_0$.
4. Thành phần đầu tiên của $\mathbf{U}_n$ chính là $u_{n+k-1}$.

##### **Ứng dụng thực tế**  
- **Giải hệ phương trình truy hồi bậc cao**: Tìm nghiệm tường minh mà không cần tính đệ quy.
- **Tính toán nhanh trong thuật toán động**: Đặc biệt trong xử lý chuỗi, số học tổ hợp.
- **Phân tích tín hiệu thời gian rời rạc**: Xác định đầu ra của hệ tuyến tính bất biến.

#### **3. Giải hệ phương trình vi phân tuyến tính**

##### **Bài toán**  
Giải hệ phương trình vi phân tuyến tính:
$$
\frac{d\mathbf{x}}{dt} = A \mathbf{x}(t)
$$
với $\mathbf{x}(0) = \mathbf{x}_0$ và $A \in M_n(\mathbb{R})$.

##### **Mô tả tổng quát**  
Nghiệm tổng quát của hệ phương trình là:
$$
\mathbf{x}(t) = e^{At} \mathbf{x}_0
$$
Việc tính $e^{At}$ trực tiếp là phức tạp. Tuy nhiên, nếu $A$ chéo hóa được (tức tồn tại $A = P D P^{-1}$), ta có:
$$
e^{At} = P e^{Dt} P^{-1}
$$
Vì $D$ là ma trận đường chéo, nên $e^{Dt}$ dễ dàng tính được bằng cách lấy lũy thừa mũ cho từng phần tử đường chéo:
$$
e^{Dt} = \begin{bmatrix}
e^{\lambda_1 t} & 0 & \cdots & 0 \\
0 & e^{\lambda_2 t} & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & e^{\lambda_n t}
\end{bmatrix}
$$

##### **Cách áp dụng**  
1. Tìm trị riêng và vector riêng để chéo hóa $A$: $A = P D P^{-1}$.
2. Tính $e^{Dt}$ bằng cách lấy mũ các phần tử đường chéo.
3. Tính $e^{At} = P e^{Dt} P^{-1}$.
4. Nghiệm là $\mathbf{x}(t) = e^{At} \mathbf{x}_0$.

##### **Ứng dụng thực tế**  
- **Mô hình tăng trưởng dân số**, **lãi kép liên tục**.
- **Cơ học cổ điển và điện tử học**: mô phỏng dao động, dao động điều hòa.
- **Mô hình hóa hệ thống động lực học**: dùng trong robot, kỹ thuật điều khiển, sinh học.

---

#### **4. Phân tích thành phần chính (PCA)**

##### **Bài toán**  
Cho tập dữ liệu gồm $m$ điểm dữ liệu trong không gian $n$ chiều, tìm các trục tọa độ mới (thành phần chính) để biểu diễn dữ liệu một cách hiệu quả nhất.

##### **Mô tả tổng quát**  
PCA yêu cầu tính trị riêng và vector riêng của **ma trận hiệp phương sai** (covariance matrix). Ma trận này luôn **đối xứng và chéo hóa được**. Trục tọa độ mới (thành phần chính) chính là các vector riêng tương ứng với trị riêng lớn nhất.

##### **Cách áp dụng**  
1. Chuẩn hóa dữ liệu.
2. Tính ma trận hiệp phương sai $C$.
3. Chéo hóa $C$ để tìm trị riêng $\lambda_i$ và vector riêng $v_i$.
4. Chọn $k$ vector riêng tương ứng với $k$ trị riêng lớn nhất.
5. Biểu diễn dữ liệu theo hệ trục mới.

##### **Ứng dụng thực tế**  
- **Giảm chiều dữ liệu** mà vẫn giữ được thông tin quan trọng.
- **Nén ảnh**, **phát hiện khuôn mặt**, **giảm nhiễu dữ liệu**.
- **Tiền xử lý dữ liệu** cho các thuật toán học máy.

---

#### **5. Học máy – Giảm chiều dữ liệu**

##### **Bài toán**  
Cho một tập dữ liệu lớn với nhiều đặc trưng (features), làm thế nào để giảm số chiều mà vẫn giữ lại thông tin quan trọng?

##### **Mô tả tổng quát**  
Các kỹ thuật học máy hiện đại (như PCA, LDA) đều dựa trên ý tưởng chéo hóa ma trận để tìm cơ sở mới (eigenvectors) giúp biểu diễn dữ liệu hiệu quả hơn. Đặc biệt, trong không gian trị riêng, các đặc trưng thường **trực giao** với nhau.

##### **Cách áp dụng**  
1. Xây dựng ma trận hiệp phương sai $C$.
2. Chéo hóa $C$ để tìm trị riêng và vector riêng.
3. Chọn một tập con vector riêng ứng với trị riêng lớn nhất.
4. Chiếu dữ liệu ban đầu lên không gian mới để huấn luyện mô hình học máy.

##### **Ứng dụng thực tế**  
- **Học không giám sát**: clustering, visualization.
- **Học có giám sát**: giảm chiều trước khi phân loại.
- **Xử lý ngôn ngữ tự nhiên**: biểu diễn từ vựng (word embedding), xử lý văn bản.

---